<a href="https://colab.research.google.com/github/BaronVonBussin/Stuff/blob/main/also_trading_cool_20241217.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def get_market_data(end_date, lookback_days=100, history_days=252):
    """
    Get market data including sufficient history for turbulence calculation
    """
    total_days = lookback_days + history_days
    start_date = (pd.to_datetime(end_date) - timedelta(days=total_days * 1.5)).strftime('%Y-%m-%d')

    sectors = {
        'XLY': 'Consumer Discretionary',
        'XLP': 'Consumer Staples',
        'XLE': 'Energy',
        'XLF': 'Financials',
        'XLV': 'Healthcare',
        'XLI': 'Industrials',
        'XLB': 'Materials',
        'XLK': 'Technology',
        'XLU': 'Utilities',
        'IYR': 'Real Estate',
        'VOX': 'Communication'
    }

    # Download SPY for OHLC data
    spy_data = yf.download('SPY', start=start_date, end=end_date)

    # Download sector data for turbulence calculation
    sector_data = pd.DataFrame()
    for ticker in sectors.keys():
        df = yf.download(ticker, start=start_date, end=end_date)['Adj Close']
        sector_data[sectors[ticker]] = df.pct_change()

    return spy_data, sector_data

def calculate_turbulence(returns_df, window=252):
    """Calculate turbulence index with rolling window"""
    turbulence = pd.Series(index=returns_df.index, dtype=float)

    for t in range(window, len(returns_df)):
        hist_window = returns_df.iloc[t-window:t]
        current_returns = returns_df.iloc[t]

        mu = hist_window.mean()
        sigma = hist_window.cov()

        try:
            sigma_inv = np.linalg.inv(sigma)
            diff = current_returns - mu
            turbulence.iloc[t] = np.sqrt(diff.dot(sigma_inv).dot(diff))
        except:
            turbulence.iloc[t] = np.nan

    return turbulence

def create_visualization(spy_data, turbulence, lookback_days=100, focus_days=50):
    """Create interactive visualization"""
    # Get the last focus_days of data
    end_date = spy_data.index[-1]
    start_date = end_date - timedelta(days=lookback_days)
    plot_data = spy_data[start_date:]

    # Calculate turbulence percentiles and regimes
    turb_plot = turbulence[start_date:]
    regime = (turb_plot > turb_plot.quantile(0.75)).astype(int)

    # Create figure with secondary y-axis
    fig = make_subplots(rows=2, cols=1,
                       shared_xaxes=True,
                       vertical_spacing=0.05,
                       row_heights=[0.7, 0.3])

    # Add candlestick
    fig.add_trace(
        go.Candlestick(
            x=plot_data.index,
            open=plot_data['Open'],
            high=plot_data['High'],
            low=plot_data['Low'],
            close=plot_data['Close'],
            name='SPY'
        ),
        row=1, col=1
    )

    # Add turbulence index
    fig.add_trace(
        go.Scatter(
            x=turb_plot.index,
            y=turb_plot,
            name='Turbulence',
            line=dict(color='rgba(255, 165, 0, 0.8)', width=2)
        ),
        row=2, col=1
    )

    # Add regime shading
    for i in range(len(regime)-1):
        if regime.iloc[i] == 1:
            fig.add_vrect(
                x0=regime.index[i],
                x1=regime.index[i+1],
                fillcolor="red",
                opacity=0.2,
                layer="below",
                line_width=0,
                row=1, col=1
            )

    # Add focus period vertical line
    focus_start = end_date - timedelta(days=focus_days)
    fig.add_vline(x=focus_start, line_dash="dash", line_color="gray")

    # Update layout
    fig.update_layout(
        title='Market Analysis with Turbulence Index',
        yaxis_title='Price',
        yaxis2_title='Turbulence',
        xaxis_rangeslider_visible=False,
        height=800,
        showlegend=True,
        plot_bgcolor='white',
        paper_bgcolor='white',
        hovermode='x unified'
    )

    # Update axes
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')

    return fig

def main():
    # Set analysis period
    end_date = '2019-12-31'
    lookback_days = 100
    focus_days = 50

    # Get data
    print("Downloading market data...")
    spy_data, sector_data = get_market_data(end_date, lookback_days)

    # Calculate turbulence
    print("Calculating turbulence index...")
    turbulence = calculate_turbulence(sector_data)

    # Create visualization
    print("Creating visualization...")
    fig = create_visualization(spy_data, turbulence, lookback_days, focus_days)

    # Show plot
    fig.show()

    return spy_data, turbulence, fig

if __name__ == "__main__":
    spy_data, turbulence, fig = main()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Calculating turbulence index...
Creating visualization...


In [12]:
# Download recent SPY data
spy = yf.download('SPY', start='2019-03-01', end='2019-12-30')

# Reset column names
spy.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']

# Create OHLC chart using plotly
fig = go.Figure(data=[go.Candlestick(
    x=spy.index,
    open=spy['Open'],
    high=spy['High'],
    low=spy['Low'],
    close=spy['Close'],
    name='SPY'
)])

# Update the layout
fig.update_layout(
    title='SPY OHLC Chart - 2023-2024',
    yaxis_title='Price (USD)',
    xaxis_rangeslider_visible=False,
    height=900,
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Configure x-axis to show only trading days
fig.update_xaxes(
    showgrid=True,
    gridwidth=1,
    gridcolor='LightGray',
    rangebreaks=[
        dict(bounds=["sat", "mon"]),  # hide weekends
        dict(values=["2023-01-16", "2023-02-20", "2023-04-07", "2023-05-29",
                    "2023-06-19", "2023-07-04", "2023-09-04", "2023-11-23",
                    "2023-12-25", "2024-01-01", "2024-01-15", "2024-02-19"])  # major US holidays
    ]
)

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')

# Show the plot
fig.show()

[*********************100%***********************]  1 of 1 completed
